In [1]:
import os
import sys
import git
import pathlib

import random
import time


import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist


PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs.constants import model_seeds, error_seeds
from libs import utils
from libs.fitnessfns import ff_mnist32_cnn_ERR, eval_mnist32_cnn_ERR
from libs.ga.ga_utils import extract_best_worst_chromosomes
from libs.utils import segregate_dataset_4Q_ent

In [2]:
# set seed
this_seed = 84376
tf.random.set_seed(this_seed)
np.random.seed(this_seed)
random.seed(this_seed)
os.environ['PYTHONHASHSEED']=str(this_seed)

In [3]:
# Tensorflow Limit GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        

In [4]:
# Use only test images
# Prepare dataset
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0
all_images =np.concatenate([train_images, test_images], axis=0)
all_labels =np.concatenate([train_labels, test_labels], axis=0)
all_images = np.expand_dims(all_images, axis=-1)
# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
all_images = tf.image.resize(all_images, [image_x_size, image_y_size]).numpy()

In [5]:
# Load test set
# Testing with only im_num images
im_num = 128*40#128*40=5120 #[128*78 = 9984]
(qent1_images, qent1_labels,
qent2_images, qent2_labels,
qent3_images, qent3_labels,
qent4_images, qent4_labels) = segregate_dataset_4Q_ent(all_images, all_labels, half_size=int(image_x_size/2))

qent1_images = qent1_images[-im_num:]
qent1_labels = qent1_labels[-im_num:]

qent2_images = qent2_images[-im_num:]
qent2_labels = qent2_labels[-im_num:]

qent3_images = qent3_images[-im_num:]
qent3_labels = qent3_labels[-im_num:]

qent4_images = qent4_images[-im_num:]
qent4_labels = qent4_labels[-im_num:]

q1_test_set = (qent1_images, qent1_labels)
q2_test_set = (qent2_images, qent2_labels)
q3_test_set = (qent3_images, qent3_labels)
q4_test_set = (qent4_images, qent4_labels)
all_test_set = (all_images[-im_num:], all_labels[-im_num:])

In [6]:
testset_type_dict= {"q1":q1_test_set, 
                    "q2":q2_test_set, 
                    "q3":q3_test_set, 
                    "q4":q4_test_set, 
                    "all":all_test_set}

In [7]:
# model_seeds = model_seeds[0:3]
# error_seeds = [error_seeds[0],error_seeds[2]]
model_seeds = model_seeds[0:3]
error_seeds = [error_seeds[0], error_seeds[2]]

In [8]:
error_profile_tag_list = []
ERR_PARAM_list = [-1]
MAX_FAULT_PROB_LIST = [5E-3]
error_layer_list = ["c0"]

for MAX_FAULT_PROB in MAX_FAULT_PROB_LIST:
    for seed in error_seeds:
        err_tag = "LIM_" + f"{int(MAX_FAULT_PROB*1E3):02d}" + "-" +str(seed)
        error_profile_tag_list.append(err_tag)

In [9]:
def eval_error_injected_with_shuffle(model, 
                                      layer_error_profiles, 
                                      ERR_PARAM,
                                      shuffle_orders,
                                      test_set):
    # intialize to None
    error_profile_c0=None
    error_profile_h0=None
    error_profile_h1=None
    error_profile_h2=None
    error_profile_op=None

    error_profile_c0 = layer_error_profiles["c0"]
    error_profile_h0 = layer_error_profiles["h0"]
    error_profile_h1 = layer_error_profiles["h1"]
    error_profile_h2 = layer_error_profiles["h2"]
    error_profile_op = layer_error_profiles["op"]

    
    # intialize to None
    shuffle_order_c0 = None
    shuffle_order_h0 = None
    shuffle_order_h1 = None
    shuffle_order_h2 = None
    shuffle_order_op = None
                    
    shuffle_order_c0 = shuffle_orders["c0"]
    shuffle_order_h0 = shuffle_orders["h0"]
    shuffle_order_h1 = shuffle_orders["h1"]
    shuffle_order_h2 = shuffle_orders["h2"]
    shuffle_order_op = shuffle_orders["op"]
        
    
    avg, std = eval_mnist32_cnn_ERR(model,
                                    error_profile_c0,
                                    error_profile_h0,
                                    error_profile_h1,
                                    error_profile_h2,
                                    error_profile_op,
                                    ERR_PARAM,
                                    shuffle_order_c0,
                                    shuffle_order_h0,
                                    shuffle_order_h1,
                                    shuffle_order_h2,
                                    shuffle_order_op,
                                    test_set)
        
    return avg

In [10]:
def reset_dict_values():
    dummy_dict =  {"c0":None,
             "h0":None,
             "h1":None,
             "h2":None,
             "op":None}
    return dummy_dict

In [11]:
dataset_seg_type_list = ["qent1","qent2","qent3","qent4", "all"]

In [12]:
model_type = "mnist32-cnn_1024_256_64"
results = {}
results_filename = 'evaluate_ga_optim-mnist32-cnn--qent-c0--LIM_05-ERR_-1.npy'

for testset_type in testset_type_dict.keys():
    print()
    print("*"*50)
    print("TESTSET: ",testset_type)
    results[testset_type] = {}
    test_set = testset_type_dict[testset_type]
    for dataset_seg_type in dataset_seg_type_list:
        print("SEG-OPTIM TYPE:\t", dataset_seg_type)
        results[testset_type][dataset_seg_type] = {}
        # one layer at a time
        for model_seed in model_seeds:#model_seeds[0:2]:
            results[testset_type][dataset_seg_type][model_seed] = {}
            # Get model
            model_instance = model_type + "-" + str(model_seed)
            dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
            model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
            model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
            model_filename = model_instance + ".h5"
            model_file = pathlib.Path(model_folder/ model_filename)
            # Load model
            model = tf.keras.models.load_model(model_file)
            print(model_instance)

            for error_profile_tag in error_profile_tag_list:
                print(error_profile_tag)
                results[testset_type][dataset_seg_type][model_seed][error_profile_tag] = {}
                # Load error profile
                error_profile_folder = pathlib.Path(PROJ_ROOT_PATH / "error_profiles")
                error_profile_filename = error_profile_tag + ".npy"
                error_profile_file = pathlib.Path(error_profile_folder/ error_profile_filename)
                error_profile = np.load(error_profile_file)

                for ERR_PARAM in ERR_PARAM_list:
                    # print("ERR_PARAM:\t", ERR_PARAM)
                    results[testset_type][dataset_seg_type][model_seed][error_profile_tag][ERR_PARAM]={}
                    for error_layer in error_layer_list:
                        # print("Layer: ", error_layer)
                        results[testset_type][dataset_seg_type][model_seed][error_profile_tag][ERR_PARAM][error_layer]={}
                        # Reset error profiles and shuffle orders for all layers
                        error_profiles = reset_dict_values()
                        shuffle_orders = reset_dict_values()
                        # Inject error
                        error_profiles[error_layer] = error_profile
                        # Get chromosomes
                        best, worst = extract_best_worst_chromosomes(dataset_seg_type, # (all, qmean1, qent2)
                                                                           error_layer, 
                                                                           ERR_PARAM, 
                                                                           model_instance, 
                                                                           error_profile_tag, 
                                                                           this_seed)
                        # using best chromosome
                        shuffle_orders[error_layer] = best
                        eval_acc  = eval_error_injected_with_shuffle(model, 
                                                                      error_profiles, 
                                                                      ERR_PARAM,
                                                                      shuffle_orders,
                                                                      test_set)
                        results[testset_type][dataset_seg_type][model_seed][error_profile_tag][ERR_PARAM][error_layer]["ga_optim"] = eval_acc
                        np.save(results_filename, results) 
                        print("GA ACC: ", eval_acc)
                print()
            print()


**************************************************
TESTSET:  q1
SEG-OPTIM TYPE:	 qent1
mnist32-cnn_1024_256_64-1023
LIM_05-2188
GA ACC:  0.9173900462962963

LIM_05-4981
GA ACC:  0.8411458333333334


mnist32-cnn_1024_256_64-2920
LIM_05-2188
GA ACC:  0.9370659722222222

LIM_05-4981
GA ACC:  0.9360532407407408


mnist32-cnn_1024_256_64-3824
LIM_05-2188
GA ACC:  0.8661747685185185

LIM_05-4981
GA ACC:  0.8090277777777777


SEG-OPTIM TYPE:	 qent2
mnist32-cnn_1024_256_64-1023
LIM_05-2188
GA ACC:  0.8990162037037037

LIM_05-4981
GA ACC:  0.8641493055555557


mnist32-cnn_1024_256_64-2920
LIM_05-2188
GA ACC:  0.9463252314814814

LIM_05-4981
GA ACC:  0.9293981481481483


mnist32-cnn_1024_256_64-3824
LIM_05-2188
GA ACC:  0.8388310185185185

LIM_05-4981
GA ACC:  0.8190104166666666


SEG-OPTIM TYPE:	 qent3
mnist32-cnn_1024_256_64-1023
LIM_05-2188
GA ACC:  0.9220196759259259

LIM_05-4981
GA ACC:  0.8611111111111112


mnist32-cnn_1024_256_64-2920
LIM_05-2188
GA ACC:  0.9461805555555555

LIM_05-4981
